
Data Mining - MSDS 7331 - Thurs 6:30, Summer 2016 Team 3 (AKA Team Super Awesome): Sal Melendez, Rahn Lieberman, Thomas Rogers
Github page: https://github.com/RahnL/DataScience-SMU/tree/master/DataMining
Data Mining - Mini Lab 1
Import the data

In [10]:
import pandas as pd
import numpy as np
from pandas import DataFrame

df = pd.read_csv("LLCP2014XPT.txt", sep="\t", encoding = "ISO-8859-1")

C:\Users\trogers\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


To start, lets try to predict Age based on general Health (GENHLTH), income and whether they are a smoker.
We will do some cleanup first:

In [16]:
df_reduced = df[['_AGE80','_INCOMG', 'GENHLTH', '_SMOKER3']]
df_reduced['health'] = pd.cut(df_reduced.GENHLTH,[0,2,5,9],3,labels=[0,1,2]) # this creates a new variable by categorizing GENHLTH into three categories
df_reduced.head()

df_reduced.describe().transpose()

df_reduced.info()

df_reduced.dropna(inplace=True) # this drops those empty variables

C:\Users\trogers\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464664 entries, 0 to 464663
Data columns (total 5 columns):
_AGE80      464664 non-null int64
_INCOMG     464664 non-null int64
GENHLTH     464660 non-null float64
_SMOKER3    464664 non-null int64
health      464660 non-null category
dtypes: category(1), float64(1), int64(3)
memory usage: 14.6 MB


C:\Users\trogers\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Training And Testing
Threefold cross validation, using 80% of data as training.

In [17]:
from sklearn.cross_validation import ShuffleSplit

#... setup x, y
#if '_Age80' in df_reduced:
y = df_reduced['health'].values # get the labels we want...changed to the categorical version of health
del df_reduced['health'] # get rid of the class label
X = df_reduced.values # use everything else to predict!

# do the cross validation
num_cv_iterations = 3
num_instances = len(y)
cv_object = ShuffleSplit(n=num_instances,
                         n_iter=num_cv_iterations,
                         test_size  = 0.2)
                         
print (cv_object)

ShuffleSplit(464660, n_iter=3, test_size=0.2, random_state=None)


In [18]:
# run logistic regression and vary some parameters
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt

# first we create a reusable logisitic regression object
#   here we can setup the object with different learning parameters and constants
lr_clf = LogisticRegression(penalty='l2', C=1.0, class_weight=None) # get object

# now we can use the cv_object that we setup before to iterate through the 
#    different training and testing sets. Each time we will reuse the logisitic regression 
#    object, but it gets trained on different data each time we use it.

iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in cv_object: 
    # I will create new variables here so that it is more obvious what 
    # the code is doing (you can compact this syntax and avoid duplicating memory,
    # but it makes this code less readable)
    X_train = X[train_indices]
    y_train = y[train_indices]
    
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    # train the reusable logisitc regression model on the training data
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions

    # now let's get the accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print ("====Iteration",iter_num," ====")
    print ("accuracy", acc) 
    print ("confusion matrix\n",conf)
    iter_num+=1

====Iteration 0  ====
accuracy 0.996384453149
confusion matrix
 [[46739     0     0]
 [    0 45857     0]
 [    0   336     0]]
====Iteration 1  ====
accuracy 0.996761072612
confusion matrix
 [[46837     0     0]
 [    0 45794     0]
 [    0   301     0]]
====Iteration 2  ====
accuracy 0.996481298153
confusion matrix
 [[46660     0     0]
 [    0 45945     0]
 [    0   327     0]]
